In [ ]:
import pypsa
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
n = pypsa.Network("../../resources/validation_2023/networks/base_s_39__2190seg_2025.nc")

In [ ]:
n.generators.query("carrier == 'coal'").marginal_cost.unique()

In [ ]:
coal_SRMC = (
    n.links.query("carrier in ['coal']").marginal_cost + (n.generators.query("carrier == 'coal'").marginal_cost.unique()[0]/n.links.query("carrier in ['coal']").efficiency)
).values
coal_cap = (n.links.query("carrier in ['coal']").p_nom * n.links.query("carrier in ['coal']").efficiency).values

lignite_SRMC = (
    n.links.query("carrier in ['lignite']").marginal_cost + (n.generators.query("carrier == 'lignite'").marginal_cost.unique()[0]/n.links.query("carrier in ['lignite']").efficiency)
).values
lignite_cap = (n.links.query("carrier in ['lignite']").p_nom * n.links.query("carrier in ['lignite']").efficiency).values

gas_SRMC = (
    n.links.query("carrier in ['OCGT', 'CCGT']").marginal_cost + (n.generators.query("carrier == 'gas'").marginal_cost.unique()[0]/n.links.query("carrier in ['OCGT', 'CCGT']").efficiency)
).values
gas_cap = (n.links.query("carrier in ['OCGT', 'CCGT']").p_nom * n.links.query("carrier in ['OCGT', 'CCGT']").efficiency).values


SRMCs_const = pd.DataFrame([coal_SRMC, coal_cap, gas_SRMC, gas_cap, lignite_SRMC, lignite_cap], index=["coal_SRMC", "coal_cap", "gas_SRMC", "gas_cap", "lignite_SRMC", "lignite_cap"]).T

In [ ]:
SRMCs_const

In [ ]:
n.carriers.loc[["gas", "coal", "lignite"]]

In [ ]:
plt.hist(SRMCs_const["coal_SRMC"], bins=30, alpha=0.75, color='#545454', label="Coal")
plt.hist(SRMCs_const["gas_SRMC"], bins=30, alpha=0.75, color='#e05b09', label="Gas")
plt.hist(SRMCs_const["lignite_SRMC"], bins=30, alpha=0.75, color='#826837', label="Lignite")

plt.xlabel("SRMC [€/MWh]")
plt.ylabel("Frequency")
plt.title("Distribution of SRMCs")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(SRMCs_const["coal_SRMC"], SRMCs_const["coal_cap"]/1e3, "o", alpha=0.75, color='#545454', label="Coal")
plt.plot(SRMCs_const["gas_SRMC"], SRMCs_const["gas_cap"]/1e3, "o", alpha=0.5, color='#e05b09', label="Gas")
plt.plot(SRMCs_const["lignite_SRMC"], SRMCs_const["lignite_cap"]/1e3, "o", alpha=0.75, color='#826837', label="Lignite")

plt.xlabel("SRMC [€/MWh]")
plt.ylabel("Capacity [GW]")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
effs = n.links.query("carrier == 'coal'").efficiency
coal_SRMC_t = pd.DataFrame(
    {
        link: n.links.at[link, "marginal_cost"] + n.generators_t.marginal_cost["EU coal"] / effs.at[link]
        for link in effs.index
    },
    index=n.snapshots
)

effs = n.links.query("carrier in ['OCGT', 'CCGT']").efficiency
gas_SRMC_t = pd.DataFrame(
    {
        link: n.links.at[link, "marginal_cost"] + n.generators_t.marginal_cost["EU gas"] / effs.at[link]
        for link in effs.index
    },
    index=n.snapshots
)

In [ ]:
n.carriers.loc["coal"]

In [ ]:
fig,ax=plt.subplots()

gas_SRMC_t.plot(color="#e05b09", legend=False, ax=ax)
coal_SRMC_t.plot(color="#545454", legend=False, ax=ax)

In [ ]:
n.generators.loc[["EU gas", "EU coal"]].marginal_cost

In [ ]:
n.carriers.loc[["gas", "coal"]]